# 1ra parte - Modelos supervisados

En esta primera parte se aplicó los modelos naive bayes, arboles de decisión, random forest y redes neuronales con el fin de determinar cual de estos modelos supervisados tiene mejores estimaciones para establecer si una persona es obesa teniendo en cuenta sus hábitos alimenticios y su condición física. 

En esta ocasión se empleó la base de datos de “Estimation of Obesity Levels Based On Eating Habits and Physical Condition” de la UC Irvine Machine Learning Repository, la cual incluye información sobre la estimación de los niveles de obesidad en personas en los países de México, Perú y Colombia con base a sus hábitos y condición física. Esta base está compuesta por 2111 registros y por 17 variables las cuales se describen mediante la siguiente tabla:


In [60]:
from ucimlrepo import fetch_ucirepo
estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition = fetch_ucirepo(id=544) 
estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.variables

,name,role,type,demographic,description,units,missing_values
0,Gender,Feature,Categorical,Gender,None,None,no
1,Age,Feature,Continuous,Age,None,None,no
2,Height,Feature,Continuous,None,None,None,no
3,Weight,Feature,Continuous,None,None,None,no
4,family_history_with_overweight,Feature,Binary,None,Has a family member suffered or suffers from o...,None,no
5,FAVC,Feature,Binary,None,Do you eat high caloric food frequently?,None,no
6,FCVC,Feature,Integer,None,Do you usually eat vegetables in your meals?,None,no
7,NCP,Feature,Continuous,None,How many main meals do you have daily?,None,no
8,CAEC,Feature,Categorical,None,Do you eat any food between meals?,None,no
9,SMOKE,Feature,Binary,None,Do you smoke?,None,no


Cabe resaltar que, para fines del ejercicio se excluyeron las variables Height y Weight debido a que tienen una alta correlación con la obesidad como se muestra en el índice de masa corporal (IMC) el cual es una medida de la obesidad que se determina por estas dos variables.

# Preparación de los datos

Se empieza a realizar la importación de las librerias necesarias para el ejercicio

In [49]:
# importar librerias para el proyecto
import pandas as pd
import numpy as np
import seaborn as sbn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model, tree
from sklearn.feature_selection import RFE
from sklearn import linear_model, tree
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeClassifier
from dtreeviz.trees import dtreeviz
from sklearn.model_selection import GridSearchCV
import joblib as joblib
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold
import keras.backend as K
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
#from keras.wrappers.scikit_learn import KerasRegressor
#from scikeras.wrappers import KerasClassifier


In [51]:
# Cargamos los datos  
datos = pd.read_csv("../Datos/ObesityDataSet_raw_and_data_sinthetic.csv")
datos.head(10)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II
5,Male,29.0,1.62,53.0,no,yes,2.0,3.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Automobile,Normal_Weight
6,Female,23.0,1.50,55.0,yes,yes,3.0,3.0,Sometimes,no,2.0,no,1.0,0.0,Sometimes,Motorbike,Normal_Weight
7,Male,22.0,1.64,53.0,no,no,2.0,3.0,Sometimes,no,2.0,no,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
8,Male,24.0,1.78,64.0,yes,yes,3.0,3.0,Sometimes,no,2.0,no,1.0,1.0,Frequently,Public_Transportation,Normal_Weight
9,Male,22.0,1.72,68.0,yes,yes,2.0,3.0,Sometimes,no,2.0,no,1.0,1.0,no,Public_Transportation,Normal_Weight


In [3]:
datos.describe()


,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000
mean,24.312600,1.701677,86.586058,2.419043,2.685628,2.008011,1.010298,0.657866
std,6.345968,0.093305,26.191172,0.533927,0.778039,0.612953,0.850592,0.608927
min,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,19.947192,1.630000,65.473343,2.000000,2.658738,1.584812,0.124505,0.000000
50%,22.777890,1.700499,83.000000,2.385502,3.000000,2.000000,1.000000,0.625350
75%,26.000000,1.768464,107.430682,3.000000,3.000000,2.477420,1.666678,1.000000
max,61.000000,1.980000,173.000000,3.000000,4.000000,3.000000,3.000000,2.000000


A continuación, se realizó la separición de las variables categoricas y númericas en conjunto de la separación de la base en testeo y train. Adicionalmente para la variable objetivo "NObeyesdad" se agruparon sus 6 categorias en dos, siendo el estado 1 si el individuo es obeso y 0 que no lo es.

In [19]:
rng = np.random.RandomState(seed=30) # Algo pasa con la semilla pero en X_train la variable CAEC queda sin always, por eso al ajustar el modelo no encuentra la categoría
categorical= ["Gender","family_history_with_overweight", "FAVC","SMOKE", "SCC","CAEC","CALC",'MTRANS']
numerical_columns = ["Age", "FCVC", "NCP", "CH2O", "FAF","TUE"]
X = datos[numerical_columns +categorical]
y = datos["NObeyesdad"]
y = pd.DataFrame(datos['NObeyesdad'].map({'Normal_Weight': 0, 'Insufficient_Weight': 0, 'Overweight_Level_I':0,'Overweight_Level_II':0, 'Obesity_Type_I':1,'Obesity_Type_II':1, 'Obesity_Type_III':1}))
yname = ["NObeyesdad"]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=54)

## Preprocessing


Se aplicó la función de ColumnTransformer para llevar a cabo la transformación de los datos. En el caso de las variables categóricas, se empleó la función OneHotEncoder para su conversión a formato numérico. En cuanto a las variables numéricas, se llevó a cabo la escalación utilizando la función StandardScaler.

In [22]:
categorical_encoder = OneHotEncoder(handle_unknown='ignore') 
numerical_pipe = StandardScaler()

preprocessing = ColumnTransformer(
    [
        ("codificacion de categoricas",categorical_encoder, categorical),
        ("Transformación de numéricas", numerical_pipe, numerical_columns)
    ],
    verbose_feature_names_out=False,
)


A continuación, se realiza la estimación de los modelos supervisados y se eligira cual es el mejor para deterinar si los individuos son obesos.

# Logistic Model

Se implementó el modelo logístico mediante las funciones Pipeline y GridSearchCV con el objetivo de generar múltiples modelos logísticos y seleccionar el que obtenga el mejor rendimiento en términos de precisión (accuracy).

In [25]:
logit = LogisticRegression(random_state=30)
modelo_logit = Pipeline(
    [
    ("preprocesamiento", preprocessing),
    ("Classifier", logit),  
    ]
)

In [26]:
k_fold = StratifiedKFold(5, shuffle=True, random_state=30)
param_grid = {'Classifier__penalty': ['l1', 'l2'],           # Tipo de penalización
              'Classifier__C': [0.001, 0.01, 0.1, 1, 10],    # Parámetro de regularización inversa (más pequeño es más fuerte)
              'Classifier__max_iter': [100, 200, 300],        # Número máximo de iteraciones
              'Classifier__fit_intercept': [True, False],     # Ajustar o no la intercepción
              'Classifier__class_weight': [None, 'balanced'] } # Manejo de clases desbalanceadas}

best_log = GridSearchCV(modelo_logit, param_grid, cv=5, scoring='accuracy')
best_log.fit(X_train, y_train)

c:\Users\LauraCuestas\anaconda3\envs\tensor_310\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\LauraCuestas\anaconda3\envs\tensor_310\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\LauraCuestas\anaconda3\envs\tensor_310\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\LauraCuestas\anaconda3\envs\tensor_310\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-v

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocesamiento',
                                        ColumnTransformer(transformers=[('codificacion '
                                                                         'de '
                                                                         'categoricas',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Gender',
                                                                          'family_history_with_overweight',
                                                                          'FAVC',
                                                                          'SMOKE',
                                                                          'SCC',
                                                                          'CAEC',
                                                                          'CALC',
                                                                          'MTRANS']),
                                                                        ('Transformación '
                                                                         'de '
                                                                         'numéricas',
                                                                         StandardScaler(),
                                                                         ['Age',
                                                                          'FCVC',
                                                                          'NCP',
                                                                          'CH2O',
                                                                          'FAF',
                                                                          'TUE'])],
                                                          verbose_feature_names_out=False)),
                                       ('Classifier',
                                        LogisticRegression(random_state=30))]),
             param_grid={'Classifier__C': [0.001, 0.01, 0.1, 1, 10],
                         'Classifier__class_weight': [None, 'balanced'],
                         'Classifier__fit_intercept': [True, False],
                         'Classifier__max_iter': [100, 200, 300],
                         'Classifier__penalty': ['l1', 'l2']},
             scoring='accuracy')

In [29]:
best_log.best_params_

{'Classifier__C': 10,
 'Classifier__class_weight': 'balanced',
 'Classifier__fit_intercept': True,
 'Classifier__max_iter': 200,
 'Classifier__penalty': 'l2'}

# Arbol de decisión

Se aplicó el modelo logístico utilizando las funciones Pipeline y GridSearchCV, con la finalidad de crear diversos modelos logísticos y elegir aquel que logre el mejor desempeño en cuanto a precisión (accuracy).

In [30]:
arbol = DecisionTreeClassifier(random_state=30)
modelo_arbol = Pipeline(
    [
    ("preprocesamiento", preprocessing),
    ("Classifier", arbol),  
    ]
)

In [31]:
k_fold = StratifiedKFold(5, shuffle=True, random_state=30)
param_grid = {"Classifier__criterion": ['entropy', 'gini','log_loss'],
              "Classifier__max_depth": range(3, 5),
              "Classifier__min_samples_leaf": range(2, 10)}

best_tree = GridSearchCV(modelo_arbol, param_grid, cv=k_fold, scoring='accuracy', n_jobs=-1)
best_tree.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=30, shuffle=True),
             estimator=Pipeline(steps=[('preprocesamiento',
                                        ColumnTransformer(transformers=[('codificacion '
                                                                         'de '
                                                                         'categoricas',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Gender',
                                                                          'family_history_with_overweight',
                                                                          'FAVC',
                                                                          'SMOKE',
                                                                          'SCC',
                                                                          'CAEC',
                                                                          'CALC',
                                                                          'MTRANS']),
                                                                        ('Transformación '
                                                                         'de '
                                                                         'numéricas',
                                                                         StandardScaler(),
                                                                         ['Age',
                                                                          'FCVC',
                                                                          'NCP',
                                                                          'CH2O',
                                                                          'FAF',
                                                                          'TUE'])],
                                                          verbose_feature_names_out=False)),
                                       ('Classifier',
                                        DecisionTreeClassifier(random_state=30))]),
             n_jobs=-1,
             param_grid={'Classifier__criterion': ['entropy', 'gini',
                                                   'log_loss'],
                         'Classifier__max_depth': range(3, 5),
                         'Classifier__min_samples_leaf': range(2, 10)},
             scoring='accuracy')

In [32]:
best_tree.best_params_

{'Classifier__criterion': 'gini',
 'Classifier__max_depth': 4,
 'Classifier__min_samples_leaf': 5}

In [33]:
print(best_tree.best_score_, best_tree.score(X_test,y_test))

0.7751207922373518 0.7121212121212122


Random Forest

In [35]:
# Random Forest
modelo=RandomForestClassifier(random_state=30)
modelo_rf = Pipeline(
    [
    ("preprocesamiento", preprocessing),
    ("clasificador", modelo),
    ]
)


In [36]:
k_fold = StratifiedKFold(5, shuffle=True, random_state=30)
param_grid = {'clasificador__criterion': ['entropy', 'log_loss', 'gini'],
              'clasificador__max_depth': range(3, 10),
              'clasificador__min_samples_leaf': range(2, 10)}
best_rf = GridSearchCV(modelo_rf, param_grid, scoring='accuracy', n_jobs=-1)
best_rf.fit(X_train, y_train)

c:\Users\LauraCuestas\anaconda3\envs\tensor_310\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(estimator=Pipeline(steps=[('preprocesamiento',
                                        ColumnTransformer(transformers=[('codificacion '
                                                                         'de '
                                                                         'categoricas',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Gender',
                                                                          'family_history_with_overweight',
                                                                          'FAVC',
                                                                          'SMOKE',
                                                                          'SCC',
                                                                          'CAEC',
                                                                          'CALC',
                                                                          'MTRANS']),
                                                                        ('Transformación '
                                                                         'de '
                                                                         'numéricas',
                                                                         StandardScaler(),
                                                                         ['Age',
                                                                          'FCVC',
                                                                          'NCP',
                                                                          'CH2O',
                                                                          'FAF',
                                                                          'TUE'])],
                                                          verbose_feature_names_out=False)),
                                       ('clasificador',
                                        RandomForestClassifier(random_state=30))]),
             n_jobs=-1,
             param_grid={'clasificador__criterion': ['entropy', 'log_loss',
                                                     'gini'],
                         'clasificador__max_depth': range(3, 10),
                         'clasificador__min_samples_leaf': range(2, 10)},
             scoring='accuracy')

In [42]:
best_rf.best_params_

{'clasificador__criterion': 'entropy',
 'clasificador__max_depth': 9,
 'clasificador__min_samples_leaf': 2}

In [43]:
print(best_rf.best_score_, best_rf.score(X_test,y_test))

0.8926266821067763 0.8977272727272727


Naive Bayes

In [44]:
gnb=GaussianNB()
modelo_GNB = Pipeline(
    [
    ("preprocesamiento", preprocessing),
    ("Classifier", gnb),  
    ]
)

k_fold = StratifiedKFold(5, shuffle=True, random_state=54)
param_grid = {}
#param_grid ={}
best_GNB = GridSearchCV(modelo_GNB, param_grid, scoring='accuracy', n_jobs=-1)
best_GNB.fit(X_train, y_train)


c:\Users\LauraCuestas\anaconda3\envs\tensor_310\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=Pipeline(steps=[('preprocesamiento',
                                        ColumnTransformer(transformers=[('codificacion '
                                                                         'de '
                                                                         'categoricas',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Gender',
                                                                          'family_history_with_overweight',
                                                                          'FAVC',
                                                                          'SMOKE',
                                                                          'SCC',
                                                                          'CAEC',
                                                                          'CALC',
                                                                          'MTRANS']),
                                                                        ('Transformación '
                                                                         'de '
                                                                         'numéricas',
                                                                         StandardScaler(),
                                                                         ['Age',
                                                                          'FCVC',
                                                                          'NCP',
                                                                          'CH2O',
                                                                          'FAF',
                                                                          'TUE'])],
                                                          verbose_feature_names_out=False)),
                                       ('Classifier', GaussianNB())]),
             n_jobs=-1, param_grid={}, scoring='accuracy')

In [45]:
print(best_GNB.best_score_, best_GNB.score(X_test,y_test))

0.7662720121391208 0.7481060606060606


In [46]:
X_train.head()

,Age,FCVC,NCP,CH2O,FAF,TUE,Gender,family_history_with_overweight,FAVC,SMOKE,SCC,CAEC,CALC,MTRANS
1532,22.705772,3.000000,2.527510,1.645338,1.025438,0.000000,Male,yes,yes,no,no,Sometimes,Sometimes,Public_Transportation
1046,23.320120,2.784471,2.122545,2.984153,2.164472,0.203978,Male,yes,yes,no,no,Sometimes,Sometimes,Public_Transportation
1726,29.791101,2.654792,1.077331,1.345407,0.327418,0.386861,Male,yes,yes,no,no,Sometimes,no,Public_Transportation
394,17.000000,2.000000,3.000000,1.000000,0.000000,1.000000,Female,yes,yes,no,no,Sometimes,no,Public_Transportation
586,18.000000,1.031149,3.000000,1.963628,0.028202,1.000000,Female,no,yes,no,no,Sometimes,Sometimes,Public_Transportation


Redes Neuronales

In [48]:
red = Sequential()
red.add(Dense(1, input_shape=(28,) ,activation='softmax'))
red.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


modelo_red = Pipeline(
    [
    ("preprocesamiento", preprocessing),
    ("Classifier", red),
    ]
)

modelo_red.fit(X_train, y_train)


c:\Users\LauraCuestas\anaconda3\envs\tensor_310\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\LauraCuestas\anaconda3\envs\tensor_310\Lib\site-packages\keras\src\losses\losses.py:22: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 865us/step - accuracy: 0.4575 - loss: 0.0000e+00 


Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('codificacion de categoricas',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Gender',
                                                   'family_history_with_overweight',
                                                   'FAVC', 'SMOKE', 'SCC',
                                                   'CAEC', 'CALC', 'MTRANS']),
                                                 ('Transformación de numéricas',
                                                  StandardScaler(),
                                                  ['Age', 'FCVC', 'NCP', 'CH2O',
                                                   'FAF', 'TUE'])],
                                   verbose_feature_names_out=False)),
                ('Classifier', <Sequential name=sequential_1, built=True>)])

In [52]:
red_neuronal = MLPClassifier()
rn_mlp = Pipeline(
    [
    ("preprocessing",preprocessing),
    ("Classifier",red_neuronal)    
    ]
)

In [58]:
k_fold = StratifiedKFold(5, shuffle=True, random_state=30)
param_grid ={ "Classifier__hidden_layer_sizes":[(10,), (20,), (30,)],
             "Classifier__activation":['identity','logistic','tanh','relu'],
             "Classifier__solver":['lbfgs','sgd','adam'],
             "Classifier__alpha":[0.0001, 0.001, 0.01, 0.1],
             "Classifier__learning_rate":['constant','invscaling','adaptative']}


best_neuralnetwork = GridSearchCV(rn_mlp, param_grid, scoring='accuracy', n_jobs=-1)
best_neuralnetwork.fit(X_train, y_train)

c:\Users\LauraCuestas\anaconda3\envs\tensor_310\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
720 fits failed out of a total of 2160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
196 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\LauraCuestas\anaconda3\envs\tensor_310\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\LauraCuestas\anaconda3\envs\tensor_310\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\LauraCuestas\anaconda3\envs\t

GridSearchCV(estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('codificacion '
                                                                         'de '
                                                                         'categoricas',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Gender',
                                                                          'family_history_with_overweight',
                                                                          'FAVC',
                                                                          'SMOKE',
                                                                          'SCC',
                                                                          'CAEC',
                                                                          'CALC',
                                                                          'MTRANS']),
                                                                        ('Transformación '
                                                                         'de '
                                                                         'numéricas',
                                                                         StandardScaler(),
                                                                         ['Age',
                                                                          'FCVC',
                                                                          'NCP',
                                                                          'CH2O',
                                                                          'FAF',
                                                                          'TUE'])],
                                                          verbose_f...e)),
                                       ('Classifier', MLPClassifier())]),
             n_jobs=-1,
             param_grid={'Classifier__activation': ['identity', 'logistic',
                                                    'tanh', 'relu'],
                         'Classifier__alpha': [0.0001, 0.001, 0.01, 0.1],
                         'Classifier__hidden_layer_sizes': [(10,), (20,),
                                                            (30,)],
                         'Classifier__learning_rate': ['constant', 'invscaling',
                                                       'adaptative'],
                         'Classifier__solver': ['lbfgs', 'sgd', 'adam']},
             scoring='accuracy')

In [59]:
print(best_neuralnetwork.best_score_, best_neuralnetwork.score(X_test,y_test))

0.9065147945533681 0.8977272727272727


In [55]:
best_neuralnetwork.best_params_

{'Classifier__activation': 'logistic',
 'Classifier__alpha': 0.1,
 'Classifier__hidden_layer_sizes': (30,),
 'Classifier__learning_rate': 'invscaling',
 'Classifier__solver': 'lbfgs'}

Fin del código

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier, KerasRegressor


model = Sequential()
model.add(Dense(units=4, input_shape=(2,), activation='sigmoid'))
model.add(Dense(units=4,  activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Construye el Pipeline
MO = Pipeline([
    ('preprocessing', preprocessing),
    ('Classifier', model)
])

# Define los parámetros a ajustar
param_grid = {
    'optimizer': ['adam', 'sgd'],
    'units': [32, 64, 128],
    'activation': ['relu', 'tanh', 'sigmoid']
}

# Configura la búsqueda de hiperparámetros utilizando GridSearchCV
grid = GridSearchCV(estimator=MO, param_grid=param_grid, cv=3, scoring='accuracy')
grid_result = grid.fit(X_train, y_train)

# Muestra los resultados
print(f"Mejores parámetros: {grid_result.best_params_}")
print(f"Precisión del modelo: {grid_result.best_score_}")


INFO:tensorflow:Assets written to: C:\Users\IP5-14~1\AppData\Local\Temp\tmpqzlrqavu\assets


INFO:tensorflow:Assets written to: C:\Users\IP5-14~1\AppData\Local\Temp\tmpqzlrqavu\assets


INFO:tensorflow:Assets written to: C:\Users\IP5-14~1\AppData\Local\Temp\tmpis05tm53\assets


INFO:tensorflow:Assets written to: C:\Users\IP5-14~1\AppData\Local\Temp\tmpis05tm53\assets


ValueError: Invalid parameter 'activation' for estimator Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('codificacion de categoricas',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Gender',
                                                   'family_history_with_overweight',
                                                   'FAVC', 'SMOKE', 'SCC',
                                                   'CAEC', 'CALC', 'MTRANS'])],
                                   verbose_feature_names_out=False)),
                ('Classifier',
                 <keras.src.engine.sequential.Sequential object at 0x000001813F9BB8B0>)]). Valid parameters are: ['memory', 'steps', 'verbose'].

In [23]:
KerasClassifier.get_params().keys()

TypeError: BaseEstimator.get_params() missing 1 required positional argument: 'self'

In [26]:
gnb2=joblib.load('bm_gnb.gz')
gnb2.fit(X_train, y_train)
#confusion_matrix(y_test, gnb.fit(X_test))
gnb2.best_params_

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\CARLOS MARIO\Anaconda3\envs\tensor_310\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\CARLOS MARIO\Anaconda3\envs\tensor_310\lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\CARLOS MARIO\Anaconda3\envs\tensor_310\lib\site-packages\sklearn\pipeline.py", line 471, in fit
    Xt = self._fit(X, y, routed_params)
  File "c:\Users\CARLOS MARIO\Anaconda3\envs\tensor_310\lib\site-packages\sklearn\pipeline.py", line 408, in _fit
    X, fitted_transformer

In [ ]:
# modelo_logistico= LogisticRegression()
gnb=GaussianNB()
# arbol=DecisionTreeClassifier(criterion='gini', min_samples_split=100, min_samples_leaf=50)
random_forest = RandomForestClassifier(random_state=42)

def red():
    modelo1 = Sequential()
    modelo1.add(Dense(units=1, input_shape=(16,), activation='sigmoid'))
    modelo1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return modelo1
red1 = red()

modelos=[modelo_logistico,gnb,arbol,random_forest, red()]
resultados=[]
for modelo in modelos:
    nombre_modelo=f'{modelo}.joblib'
    flujo = Pipeline(
        [
            ("Preprocesamiento", preprocessing),
            ("Clasificador", modelo),
        ]
    )
    joblib.dump(flujo, nombre_modelo)
    salida=joblib.dump(flujo, nombre_modelo)
    resultados.append(salida)

from sklearn.metrics import classification_report
#print(classification_report(y_test, y_pred))

In [ ]:
resultados

In [ ]:

rf=joblib.load('r_RandomForestClassifier(random_state=42).joblib')
confusion_matrix(y_test, rf.predict(X_test))
rf



In [ ]:
rf.predict(X_test)

In [ ]:
log=joblib.load('r_LogisticRegression().joblib')
confusion_matrix(y_test, log.predict(X_test))

In [ ]:
nb=joblib.load('r_GaussianNB().joblib')
confusion_matrix(y_test, nb.predict(X_test))

In [ ]:
log.fit(X_train,y_train)